In [72]:
import visual_behavior.visualization.qc.data_loading as dl
import yaml
import plotly.graph_objects as go
import time
from multiprocessing import Pool

%widescreen
%standard_imports

imported:
	os
	pandas as pd
	numpy as np
	matplotlib.pyplot as plt


In [2]:
def load_data():
    container_df = dl.build_container_df()
    filtered_container_list = dl.get_filtered_ophys_container_ids()  # NOQA F841
    container_subsets = pd.read_csv('/home/dougo/spreadsheet_map.csv')
    res = container_df.query('container_id in @filtered_container_list')
    return res.merge(container_subsets, left_on='container_id', right_on='container_id', how='left')

def load_yaml(yaml_path):
    with open(yaml_path, 'r') as stream:
        yaml_contents = yaml.safe_load(stream)

    options = []
    for k, v in yaml_contents.items():
        options.append({'label': k, 'value': v})
    return options

In [60]:
def load_container_plot_options():
    plot_definition_path = "/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/qc_plots/container_plots/plot_definitions.yml"
    container_options = load_yaml(plot_definition_path)
    return container_options

In [16]:
def get_container_plot_path(container_id, plot_type):
    qc_plot_folder = '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/qc_plots'
    container_plot_folder = os.path.join(qc_plot_folder, 'container_plots')

    plot_image_path = os.path.join(
        container_plot_folder,
        plot_type, 'container_{}.png'.format(container_id)
    )
    
    return plot_image_path

In [37]:
def does_plot_exist(container_id, plot_type):
    return os.path.exists(get_container_plot_path(container_id, plot_type))

In [55]:
def check_all_plots(container_id):
    plots = load_container_plot_options()
    d = {'container_id': container_id}
    for entry in plots:
        plot_type = entry['value']
        d.update({plot_type: does_plot_exist(container_id, plot_type)})
    return d

In [89]:
CONTAINER_TABLE = load_data().sort_values('first_acquistion_date')

def generate_plot_inventory():
    global CONTAINER_TABLE
    container_table = CONTAINER_TABLE
    plots = load_container_plot_options()
    list_of_dicts = []
    for container_id in container_table['container_id'].values:
        d = {'container_id': container_id}
        for entry in plots:
            plot_type = entry['value']
            d.update({plot_type: os.path.exists(get_container_plot_path(container_id, plot_type))})
        list_of_dicts.append(d)
    return pd.DataFrame(list_of_dicts).set_index('container_id').sort_index()

In [92]:
%%time
plot_inventory = generate_plot_inventory()

CPU times: user 44.4 ms, sys: 8.7 ms, total: 53.1 ms
Wall time: 53 ms


In [110]:
def make_plot_inventory_heatmap(plot_inventory):
    fig = go.Figure(
        data=go.Heatmap(
            z=plot_inventory.values.astype(int),
            x=plot_inventory.columns,
            y=plot_inventory.index,
            hoverongaps=True,
            showscale=False,
            #         colorbar={'title':value},
            colorscale='inferno',
            xgap=3,
            ygap=3,
        )
    )

    fig.update_layout(
        autosize=False,
        width=1000,
        height=3000,
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=50,
            pad=0
        ),
        xaxis_title='ophys session',
        yaxis_title='container ID',
        title='Plot Inventory (black = missing)'
    )
    fig.update_yaxes(autorange="reversed", type='category', dtick=1)
    fig.update_xaxes(dtick=1)

    return fig

In [123]:
import datetime
timestamp = datetime.datetime.now()
timestamp_string = 'last updated on {} @ {}'.format(timestamp.strftime('%D'),timestamp.strftime('%H:%M:%S'))
timestamp_string

'last updated on 04/03/20 @ 11:30:53'

In [111]:
plot_inventory_fig = make_plot_inventory_heatmap(plot_inventory)
plot_inventory_fig.show()
plot_inventory_fig.write_html("/home/dougo/code/dougollerenshaw.github.io/figures_to_share/container_plot_inventory.html")
plot_inventory_fig.write_html("/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/qc_plots/container_plot_inventory.html")

In [12]:
for col in plot_inventory.columns:
    print(col)
    print(plot_inventory[col].value_counts())
    print()

average_images
True     261
False      3
Name: average_images, dtype: int64

average_images_movies
True     254
False     10
Name: average_images_movies, dtype: int64

average_intensity_timeseries
True     248
False     16
Name: average_intensity_timeseries, dtype: int64

behavior_metric_summary
True    264
Name: behavior_metric_summary, dtype: int64

cell_snr_by_experiment
True     247
False     17
Name: cell_snr_by_experiment, dtype: int64

dff_traces_heatmaps
True     258
False      6
Name: dff_traces_heatmaps, dtype: int64

eyetracking_sample_frames
True    264
Name: eyetracking_sample_frames, dtype: int64

FOV_average_intensity
True     235
False     29
Name: FOV_average_intensity, dtype: int64

fraction_matched_cells
True     245
False     19
Name: fraction_matched_cells, dtype: int64

lick_rasters
True     222
False     42
Name: lick_rasters, dtype: int64

max_intensity_projection
True     261
False      3
Name: max_intensity_projection, dtype: int64

motion_correction_xy_shift
